In [ ]:
%pip install dash-bootstrap-components
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc  

# --- 1. DATA PREPARATION (The "Backend") ---
df = pd.read_csv("HRDataset_v14.csv") 

df['Termd'] = pd.to_numeric(df['Termd'], errors='coerce')
turnover_by_dept = (df.groupby('Department')['Termd'].mean() * 100).reset_index()
turnover_by_dept = turnover_by_dept.rename(columns={'Termd': 'Turnover Rate (%)'})

df['Salary'] = pd.to_numeric(df['Salary'], errors='coerce')
avg_salary_dept = df.groupby('Department')['Salary'].mean().reset_index().round(2)
avg_salary_dept = avg_salary_dept.rename(columns={'Salary': 'Average Salary'})

df['EmpSatisfaction'] = pd.to_numeric(df['EmpSatisfaction'], errors='coerce')
avg_satisfaction_dept = df.groupby('Department')['EmpSatisfaction'].mean().reset_index().round(2)
avg_satisfaction_dept = avg_satisfaction_dept.rename(columns={'EmpSatisfaction': 'Average Satisfaction (1-5)'})

race_by_dept = (
    df.groupby('Department')['RaceDesc']
    .value_counts(normalize=True)
    .mul(100)
    .round(2)
    .reset_index(name='Percentage')
)

gender_by_dept = (
    df.groupby('Department')['Sex']
    .value_counts(normalize=True)
    .mul(100)
    .round(2)
    .reset_index(name='Percentage')
)

# Step 2: Creating the charts/dashboard figures
fig_turnover = px.bar(
    turnover_by_dept.sort_values(by='Turnover Rate (%)'),
    x='Department',
    y='Turnover Rate (%)',
    title='Turnover Rate (%) by Department'
)

fig_salary = px.bar(
    avg_salary_dept.sort_values(by='Average Salary'),
    x='Department',
    y='Average Salary',
    title='Average Salary by Department'
)

fig_satisfaction = px.bar(
    avg_satisfaction_dept.sort_values(by='Average Satisfaction (1-5)'),
    x='Department',
    y='Average Satisfaction (1-5)',
    title='Average Employee Satisfaction by Department'
)

# Step 3: intitalizing the dash app

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server 

# Step 4: Making the app layout 

app.layout = dbc.Container(fluid=True, children=[
    
    html.H1(
        children='People Analytics Dashboard',
        style={'textAlign': 'center', 'color': '#333', 'padding': '20px'}
    ),
    
    html.Hr(),

    # Row 1: Turnover and Salary
    dbc.Row([
        # Left Column: Turnover
        dbc.Col(width=6, children=[
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(
                        id='turnover-graph', 
                        figure=fig_turnover, 
                        style={'height': '500px'} 
                    )
                ])
            ])
        ]),
        # Right Column: Salary
        dbc.Col(width=6, children=[
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(
                        id='salary-graph', 
                        figure=fig_salary, 
                        style={'height': '500px'} 
                    )
                ])
            ])
        ])
    ], style={'padding': '10px'}),
    
    html.Hr(),

    # Row 2: Satisfaction and Diversity 
    dbc.Row([
        # Left Column: Satisfaction
        dbc.Col(width=6, children=[
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(
                        id='satisfaction-graph', 
                        figure=fig_satisfaction, 
                        style={'height': '500px'} 
                    )
                ])
            ])
        ]),
        
        # Right Column: Diversity (interactive charts)
        dbc.Col(width=6, children=[
            dbc.Card([
                dbc.CardBody([
                    html.H2(
                        children='Department Diversity Analysis',
                        style={'textAlign': 'center', 'color': '#333'}
                    ),
                    dcc.RadioItems(
                        id='diversity-selector',
                        options=[
                            {'label': 'Racial/Ethnic Diversity', 'value': 'Race'},
                            {'label': 'Gender Diversity', 'value': 'Gender'}
                        ],
                        value='Race',
                        labelStyle={'display': 'inline-block', 'marginRight': '20px'},
                        style={'textAlign': 'center', 'marginBottom': '20px'}
                    ),
                    dcc.Graph(
                        id='diversity-graph', 
                        style={'height': '500px'} # <-- FIXED HEIGHT
                    )
                ])
            ])
        ])
    ], style={'padding': '10px'})
])


# Step 5: Defining the callback (The "Interactivity")
@app.callback(
    Output('diversity-graph', 'figure'),
    Input('diversity-selector', 'value')
)
def update_diversity_graph(selected_view):
    
    if selected_view == 'Race':
        fig = px.bar(
            race_by_dept,
            x='Department',
            y='Percentage',
            color='RaceDesc',
            title='Racial/Ethnic Diversity by Department'
        )
    else:
        fig = px.bar(
            gender_by_dept,
            x='Department',
            y='Percentage',
            color='Sex',
            title='Gender Diversity by Department'
        )
    
    fig.update_layout(
        barmode='stack',
        yaxis_title='Percentage (%)',
        xaxis_title='Department',
        xaxis_tickangle=-45
    )
    
    return fig


# Step 6: Running the App
if __name__ == '__main__':
    app.run(debug=True)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
